# Шаг 1. Импорт данных для работы

In [1]:
%%time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import yaml
from bs4 import BeautifulSoup # Превращалка html в текст.

%matplotlib inline

df_sample = pd.read_csv('sampleSubmission.csv', sep = ',')
df_train = pd.read_csv('train.csv', sep = '\t')
df_test = pd.read_csv('test.csv', sep = '\t')
print('df_train ',df_train.shape)
print('df_test ',df_test.shape)

df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'test'
df = df_test.append(df_train).reset_index(drop=True)
print('df ',df.shape)

# Переводим текстовы данные из test в словарь 'Название должности' + 'Описание'   
df_name = df[['name']].to_dict()['name']
df_desc = df[['description']].to_dict()['description']

df_namedesc = {}
for key in list(df_desc.keys()):
    df_namedesc[key] = df_name[key] + ' ' + df_desc[key]
print('df_namedesc ',len(df_namedesc))

del df_desc
del df_name 
   
# del df_sample
# del df_train
# del df_test

df_train  (200000, 4)
df_test  (170179, 3)


/Users/dinar/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


df  (370179, 5)
df_namedesc  370179
CPU times: user 11.1 s, sys: 3.1 s, total: 14.2 s
Wall time: 15.3 s


# Шаг 2. Подготовка текста: очистка от тегов, оставляем только нужные слова и т.п.

Данный алгоритм ниже пройден за 4 часа.
Результаты записаны в файл 'df_namedesc_re.csv'.

In [2]:
# %%time

# import pymorphy2 # Морфологический анализатор.
# from collections import Counter # Не считать же частоты самим.
# import math # Корень квадратный.

# morph = pymorphy2.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.

# posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
# meaningfullPoSes=['ADJF', 'NOUN', 'VERB']
# def getArticleDictionary(text, needPos=None):
#     words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
#     reswords=[]
#     for w in words:
#         wordform=morph.parse(w)[0]
#         if wordform.tag.POS in meaningfullPoSes:
#             if needPos!=None:
#                 reswords.append(wordform.normal_form+posConv[wordform.tag.POS])
#             else:
#                 reswords.append(wordform.normal_form)
#     return Counter(reswords)


# df_namedesc_re = {}
# for line in df_namedesc.keys():
#     df_namedesc_re[line] = ' '.join(list(dict(getArticleDictionary(df_namedesc[line],True).items()).keys()))
# print('df_namedesc_re ',len(df_namedesc_re))
# del df_namedesc

# with open('df_namedesc_re.csv','w') as f:
#     for key in list(df_namedesc_re.keys()):
#         f.write(str(key) + '\t' + df_namedesc_re[key] + '\n')
# f.close()

Вытащим эти данные из 'df_namedesc_re.csv' обратно в словарь df_namedesc_re

In [3]:
%%time

df_namedesc_re = {}
with open('df_namedesc_re.csv','r') as f:
    for line in f:
        line = line.strip().split('\t')
        key = line[0]
        value = line[1] if len(line)>1 else ''
        df_namedesc_re[key] = value
print('df_namedesc_re ',len(df_namedesc_re))

df_namedesc_re  370179


# Шаг 3. Перевод текста в вектор для машинного обучения

In [4]:
%%time

from IPython.display import display, Math
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer #импорт декодеров текста в вектор

# раскладываем каждый текст на вектор из 'max_features' слов/столбцов
coder = CountVectorizer(encoding='utf-8',max_features=6000)

CPU times: user 289 ms, sys: 203 ms, total: 492 ms
Wall time: 671 ms


In [5]:
%%time

df_x = coder.fit_transform(df_namedesc_re.values())

df_preproc = pd.DataFrame(df_x.toarray())
df_preproc['sample'] = df['sample']
df_preproc['target'] = df['target']

df_train_preproc = df_preproc.query('sample == "train"').drop(['sample'], axis=1)
df_test_preproc = df_preproc.query('sample == "test"').drop(['sample'], axis=1)
del df_preproc

df_y_train = list(df_train_preproc['target'])
df_x_train = df_train_preproc.drop(['target'], axis = 1)
df_y_test = list(df_test_preproc['target']) # здесь неизвестные значения Nan - которые нужно найти
df_x_test = df_test_preproc.drop(['target'], axis = 1) # здесь входные параметры
del df_train_preproc
del df_test_preproc

print('df_x_train ',df_x_train.shape)
print('df_y_train ',len(df_y_train))
print('df_x_test ',df_x_test.shape,'\n')

df_x_train  (200000, 6000)
df_y_train  200000
df_x_test  (170179, 6000) 

CPU times: user 1min 10s, sys: 1min 22s, total: 2min 33s
Wall time: 3min 11s


In [6]:
%%time
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y = train_test_split(df_x_train, df_y_train, test_size=0.33, random_state=42)
print('X_train, y_train ',X_train.shape,'X_test, y ',X_test.shape)

/Users/dinar/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


X_train, y_train  (134000, 6000) X_test, y  (66000, 6000)
CPU times: user 5.99 s, sys: 11.9 s, total: 17.9 s
Wall time: 21.3 s


# Шаг 4. Модели машинного обучения

In [7]:
%%time
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

CPU times: user 71.1 ms, sys: 79.1 ms, total: 150 ms
Wall time: 220 ms


In [8]:
%%time

model3_1 = LogisticRegression()                     # инициализируем модель
model3_1.fit(X_train, y_train)                      # обучаем модель
predict = model3_1.predict(X_test)                      # делаем предсказание

cross3_1 = cross_val_score(model3_1,df_x_train,df_y_train,scoring='roc_auc', cv=10)
print('#'*10,'>>> AVG roc_auc for Train: <<<','#'*10,
      '\nmodel3_1. LogisticRegression\t\t\t\t{:.4f}'
      .format( (sum(cross3_1) / float(len(cross3_1))) ) )

########## >>> AVG roc_auc for Train: <<< ########## 
model3_1. LogisticRegression				0.9880
CPU times: user 6min 29s, sys: 6min 35s, total: 13min 4s
Wall time: 16min 11s


# Шаг 5. Запись результатов по лучшей модели.

In [9]:
%%time
predict = model3_1.predict(df_x_test)
df_predict = pd.DataFrame(list(df_test['id']))
df_predict.columns = ['id']
df_predict['target'] = pd.DataFrame(predict)
print(df_predict.head())

df_predict.to_csv('result_submission_hw04_words6000.csv', sep=',', encoding='utf-8',index=False)

       id  target
0  200000     1.0
1  200001     1.0
2  200002     1.0
3  200003     1.0
4  200004     0.0
CPU times: user 7.58 s, sys: 18.9 s, total: 26.5 s
Wall time: 32.1 s
